<a href="https://colab.research.google.com/github/beinghorizontal/Quantext/blob/main/lstm_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pandas_datareader import data
import datetime as dt
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [ ]:
# Download stock market data (e.g., Apple Inc.)
start_date = dt.datetime(2022, 1, 1)
end_date = dt.datetime(2024, 2, 28)
with open('G:/anaconda/Scripts/data/alpha_vantage/key.txt', 'r') as f:
    key = f.read()
symbol = 'AAPL'  # Ticker symbol for Apple Inc.
df = data.DataReader("AAPL", "av-daily", start=datetime(2017, 2, 9),
                    end=datetime(2024, 2, 26),
                    api_key=key)

df['Open_pct'] = df['open'].pct_change()
df['High_pct'] = df['high'].pct_change()
df['Low_pct'] = df['low'].pct_change()
df['Close_pct'] = df['close'].pct_change()

# Drop NaN values
df.dropna(inplace=True)


In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[['Open_pct', 'High_pct', 'Low_pct', 'Close_pct']])


In [ ]:
# Create sequences of length 10
sequence_length = 100
X, y = [], []
for i in range(len(scaled_data) - sequence_length):
    X.append(scaled_data[i:i + sequence_length])
    y.append(1 if df['close'].iloc[i + sequence_length] > df['close'].iloc[i + sequence_length - 1] else 0)

X = np.array(X)
y = np.array(y)


In [ ]:
# Split data into train and validation sets
split_ratio = 0.7
split_index = int(split_ratio * len(X))
X_train, y_train = X[:split_index], y[:split_index]
X_val, y_val = X[split_index:], y[split_index:]



In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(sequence_length, 4), activation='relu'))
model.add(LSTM(64, return_sequences=True, activation='relu'))  # Add an extra LSTM layer
model.add(LSTM(64, return_sequences=True, activation='relu'))  # Add an extra LSTM layer
model.add(LSTM(64, return_sequences=True, activation='relu'))  # Add an extra LSTM layer
model.add(LSTM(64, return_sequences=True, activation='relu'))  # Add an extra LSTM layer
model.add(LSTM(64))  # Last LSTM layer (no return_sequences)
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
train_loss_list = []
val_loss_list = []

# Custom training loop
num_epochs = 500
for epoch in range(num_epochs):
    # Train the model
    history = model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_val, y_val))
    train_loss = history.history['loss'][0]
    train_loss_list.append(train_loss)

    # Calculate validation loss (use your validation data)
    # val_loss = model.evaluate(X_val, y_val, verbose=0)
    # val_loss_list.append(val_loss)

    # Update the plot
    plt.clf()  # Clear the previous plot
    plt.plot(train_loss_list, label='Training Loss')
    # plt.plot(val_loss_list, label='Validation Loss')  # Uncomment for validation loss
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.pause(0.01)  # Pause to update the plot

# Show the final plot
plt.show()